<a href="https://colab.research.google.com/github/yutaro-tanaka-yt2705/ag-cgan/blob/main/0_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import gzip
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_vcf_header(filename):
    with gzip.open(filename, "rb") as fi:
        for l in fi:
            l = l.decode("utf-8")
            if l.startswith("##"):
                continue
            elif l.startswith("#"):
                return l[1:].strip().split("\t")
            else:
                raise ValueError("Something wrong in the vcf file!")

#Obtain data from 1000 Genomes. For this, we will be using Chromosome X. 
vcf_filename = '/content/drive/MyDrive/artificial_genome_project/ALL.chrX.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz'

vcf = pd.read_csv(vcf_filename, comment='#', sep="\t")
vcf.columns = get_vcf_header(vcf_filename)
print(vcf.shape)
vcf.head()

(106962, 2557)


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,X,12583,.,C,T,.,PASS,AC=97;AN=5096;DP=8520;AF=0.02;EAS_AF=0;EUR_AF=...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,X,13587,.,T,C,.,PASS,AC=198;AN=5096;DP=5147;AF=0.04;EAS_AF=0.03;EUR...,GT,0|0,...,0|0,1|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0
2,X,13590,.,T,C,.,PASS,AC=198;AN=5096;DP=5128;AF=0.04;EAS_AF=0.03;EUR...,GT,0|0,...,0|0,1|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0
3,X,13615,.,A,G,.,PASS,AC=20;AN=5096;DP=4673;AF=0;EAS_AF=0;EUR_AF=0;A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,X,13878,.,G,C,.,PASS,AC=66;AN=5096;DP=3473;AF=0.01;EAS_AF=0;EUR_AF=...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [4]:
#count number of mutations in HG00096 (0:reference allele, 1:alternative allele)
vcf.HG00096.value_counts()

0|0    98930
1|1     2915
0|1     2606
1|0     2511
Name: HG00096, dtype: int64

In [3]:
#Prepare table of variant info.
variant_info = vcf[['CHROM', 'POS', 'REF', 'ALT', 'INFO']]
variant_info['allele_count'] = variant_info.INFO.apply(lambda x: int(x.split(';')[0].replace('AC=', '')))
variant_info['allele_freq'] = variant_info.INFO.apply(lambda x: float(x.split(';')[3].replace('AF=', '')))
variant_info['eas_allele_freq'] = variant_info.INFO.apply(lambda x: float(x.split(';')[4].replace('EAS_AF=', '')))
variant_info['eur_allele_freq'] = variant_info.INFO.apply(lambda x: float(x.split(';')[5].replace('EUR_AF=', '')))
variant_info['afr_allele_freq'] = variant_info.INFO.apply(lambda x: float(x.split(';')[6].replace('AFR_AF=', '')))
variant_info['amr_allele_freq'] = variant_info.INFO.apply(lambda x: float(x.split(';')[7].replace('AMR_AF=', '')))
variant_info['sas_allele_freq'] = variant_info.INFO.apply(lambda x: float(x.split(';')[8].replace('SAS_AF=', '')))
variant_info = variant_info.drop('INFO', axis=1)
#save table to GDrive.
variant_info.to_csv('/content/drive/MyDrive/artificial_genome_project/chrX_variant_info_formatted.csv', index=False)
variant_info.head()

,CHROM,POS,REF,ALT,allele_count,allele_freq,eas_allele_freq,eur_allele_freq,afr_allele_freq,amr_allele_freq,sas_allele_freq
0,X,12583,C,T,97,0.02,0.00,0.00,0.07,0.01,0.00
1,X,13587,T,C,198,0.04,0.03,0.02,0.06,0.02,0.05
2,X,13590,T,C,198,0.04,0.03,0.02,0.06,0.02,0.05
3,X,13615,A,G,20,0.00,0.00,0.00,0.01,0.00,0.00
4,X,13878,G,C,66,0.01,0.00,0.01,0.03,0.01,0.01


In [ ]:
#format VCF file (breakdown alleles)

id_variant_list = []
for id in tqdm(vcf.columns[9:]):
  id_a = [id, 'A'] #first chromosome
  id_b = [id, 'B'] #second chromosome
  id_column = vcf[id].tolist() #get list of variants for each id
  for variant in id_column:
    variant_a = variant.split('|')[0]
    variant_b = variant.split('|')[1]
    id_a.append(variant_a)
    id_b.append(variant_b)
  
  id_variant_list.append(id_a)
  id_variant_list.append(id_b)

#adding gene ids into dataframe.
column_names = ['ID', 'pair']
column_names.extend( (vcf['CHROM'] + '_' +  vcf['POS'].astype(str) + '_' + vcf['REF'] + '_' + vcf['ALT']).tolist() )

id_variant_df = pd.DataFrame(id_variant_list)
id_variant_df.columns = column_names
id_variant_df.head()

#save table to GDrive
id_variant_df.to_csv('/content/drive/MyDrive/artificial_genome_project/chrX_vcf_formatted.csv', index=False)

100%|██████████| 2548/2548 [03:30<00:00, 12.11it/s]


---

In [ ]:
#Prepare populations tags for genomes
sample_df = pd.read_csv('/content/drive/MyDrive/arificial_genome_project/igsr-1000 genomes on grch38.tsv.tsv', delimiter = '\t')[['Sample name', 'Population code', 'Population name', 'Superpopulation code', 'Superpopulation name']]
sample_df.head()